## Homework 3: Financial Ratio Quantile Strategies
### FINM 33150 (Winter 2025) - Quantitative Trading Strategies
#### NAME: Arturo Charleston
#### Student ID: 12449584

### Introduction
In this notebook we implement a

In [1]:
# PACKAGES
import os
import nasdaqdatalink
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import datetime
import quandl
import functools
import warnings

# Global Options
warnings.filterwarnings("ignore")
pd.options.display.float_format = '{:.5f}'.format

# Set directory
home_directory = "/Users/charleston/Downloads/git_repositories/finm-qts-2025"
os.chdir(home_directory)

# Set keys
from config import QUANDL_KEY
quandl_keys = QUANDL_KEY
nasdaqdatalink.ApiConfig.api_key = quandl_keys
quandl.ApiConfig.api_key = quandl_keys

# Set dates
start_date = '2017-01-01'
end_date = '2024-06-31'



In [2]:
# FUNCTIONS
@functools.lru_cache(maxsize=1600)
def grab_quandl_table(
    table_path,
    avoid_download=False,
    replace_existing=False,
    date_override=None,
    allow_old_file=False,
    **kwargs,
):
    root_data_dir = os.path.join(home_directory, "data", "quandl_data_table_downloads")
    data_symlink = os.path.join(root_data_dir, f"{table_path}_latest.zip")
    if avoid_download and os.path.exists(data_symlink):
        print(f"Skipping any possible download of {table_path}")
        return data_symlink
    
    table_dir = os.path.dirname(data_symlink)
    if not os.path.isdir(table_dir):
        print(f'Creating new data dir {table_dir}')
        os.mkdir(table_dir)

    if date_override is None:
        my_date = datetime.datetime.now().strftime("%Y%m%d")
    else:
        my_date = date_override
    data_file = os.path.join(root_data_dir, f"{table_path}_{my_date}.zip")

    if os.path.exists(data_file):
        file_size = os.stat(data_file).st_size
        if replace_existing or not file_size > 0:
            print(f"Removing old file {data_file} size {file_size}")
        else:
            print(
                f"Data file {data_file} size {file_size} exists already, no need to download"
            )
            return data_file

    dl = quandl.export_table(
        table_path, filename=data_file, api_key=quandl_keys, **kwargs
    )
    file_size = os.stat(data_file).st_size
    if os.path.exists(data_file) and file_size > 0:
        print(f"Download finished: {file_size} bytes")
        if not date_override:
            if os.path.exists(data_symlink):
                print(f"Removing old symlink")
                os.unlink(data_symlink)
            print(f"Creating symlink: {data_file} -> {data_symlink}")
            os.symlink(
                data_file, data_symlink,
            )
    else:
        print(f"Data file {data_file} failed download")
        return
    return data_symlink if (date_override is None or allow_old_file) else "NoFileAvailable"

@functools.lru_cache(maxsize=1600)
def fetch_quandl_table(table_path, avoid_download=True, **kwargs):
    return pd.read_csv(
        grab_quandl_table(table_path, avoid_download=avoid_download, **kwargs)
    )

In [3]:
# Download data
eod_prices = fetch_quandl_table('QUOTEMEDIA/PRICES', avoid_download=True)
ZACKS_FC = fetch_quandl_table('ZACKS/FC', avoid_download=True)
ZACKS_FR = fetch_quandl_table('ZACKS/FR', avoid_download=True)
# ZACKS_MT = fetch_quandl_table('ZACKS/MT', avoid_download=True) # Master table; no needed for this analysis
ZACKS_MKTV = fetch_quandl_table('ZACKS/MKTV', avoid_download=True)
ZACKS_SHRS = fetch_quandl_table('ZACKS/SHRS', avoid_download=True)
# ZACKS_HDM = fetch_quandl_table('ZACKS/HDM', avoid_download=True) # Table with Corporate action; no needed for this analysis

Skipping any possible download of QUOTEMEDIA/PRICES
Skipping any possible download of ZACKS/FC
Skipping any possible download of ZACKS/FR
Skipping any possible download of ZACKS/MKTV
Skipping any possible download of ZACKS/SHRS


# Data Cleaning

- Filter to the dates we are interested in
- Get the tickers that have an observation in every date in our sample (1884 trading days)

In [4]:
data_tables = [ZACKS_FC, ZACKS_FR, ZACKS_MKTV, ZACKS_SHRS]

# Filter to initial date to save memory
for i, table_name in enumerate(['ZACKS_FC', 'ZACKS_FR', 'ZACKS_MKTV', 'ZACKS_SHRS']):
    table = data_tables[i]
    if 'per_end_date' in table.columns:
        table['per_end_date'] = pd.to_datetime(table['per_end_date'])
        table = table[table['per_type'] == 'Q']
        globals()[table_name] = table[table['per_end_date'] > start_date]
        


In [5]:
# End of day prices cleaning
eod_prices.query('date >= @start_date and date <= @end_date', inplace=True)
eod_prices['date'] = pd.to_datetime(eod_prices['date'])
eod_prices = eod_prices[['ticker', 'date', 'adj_close', 'adj_volume']]

# Get the valid tickers
num_dates = len(eod_prices['date'].unique())
valid_tickers = eod_prices.groupby('ticker').filter(lambda x: x['date'].nunique() == num_dates)
valid_tickers['ticker'].nunique()

4844

In [6]:
# Sort the ZACKS_MKTV data by ticker and per_end_date
ZACKS_MKTV = ZACKS_MKTV.sort_values(by=['ticker', 'per_end_date'])

# Now perform a left merge
valid_tickers_mkt = pd.merge_asof(
    valid_tickers.sort_values(by='date'), 
    ZACKS_MKTV[['ticker', 'per_end_date', 'mkt_val']].sort_values(by='per_end_date'), 
    by='ticker', 
    left_on='date', 
    right_on='per_end_date', 
    direction='backward'
).sort_values(["ticker", "date"])

valid_tickers_mkt['mkt_val'] = valid_tickers_mkt['mkt_val'].bfill()
min_mtk_cap = 1_000 # Mkt is already in thousands
valid_tickers_mkt = valid_tickers_mkt.groupby('ticker').filter(lambda x: x['mkt_val'].min() > min_mtk_cap)

# Get the number of valid tickers
valid_tickers_mkt['ticker'].nunique()



2320

In [7]:
# Remove observations in which the total debt to total equity ratio is less than 0.1
# Sort the ZACKS_FR data by ticker and per_end_date
ZACKS_FR = ZACKS_FR.sort_values(by=['ticker', 'per_end_date'])

# Now perform a left merge
valid_tickers_mkt_debt = pd.merge_asof(
    valid_tickers_mkt.sort_values(by='date'), 
    ZACKS_FR[['ticker','tot_debt_tot_equity', 'ret_invst', 'per_end_date']].sort_values(by='per_end_date'), 
    by='ticker', 
    left_on='date', 
    right_on='per_end_date', 
    direction='backward'
).sort_values(["ticker", "date"])

valid_tickers_mkt_debt['tot_debt_tot_equity'] = valid_tickers_mkt_debt['tot_debt_tot_equity'].bfill()

valid_tickers_mkt_debt = valid_tickers_mkt_debt.groupby('ticker').filter(lambda x: x['tot_debt_tot_equity'].max() > 0.1)

valid_tickers_mkt_debt.drop(columns=['per_end_date_x', 'per_end_date_y'], inplace=True)

valid_tickers_mkt_debt['ticker'].nunique()


2190

In [8]:
# Restrict to tickers that are NOT in the automotive, financial, or insurance sector (sectors 5, 13 from zacks_sector_code)
valid_tickers_mkt_debt = valid_tickers_mkt_debt[
    valid_tickers_mkt_debt['ticker'].isin(
        ZACKS_FC[~ZACKS_FC["zacks_sector_code"].isin([5, 13])]['ticker'].unique()
    )
]

valid_tickers_mkt_debt['ticker'].nunique()


978

### Calculate financial ratios

In [ ]:
ZACKS_SHRS

In [12]:
FC_columns = ['ticker', 'net_lterm_debt', 'tot_lterm_debt', 'eps_diluted_net', 'basic_net_eps', 'filing_date', 'per_end_date']

financial_ratios = pd.merge_asof(
    valid_tickers_mkt_debt.sort_values(by='date'), 
    ZACKS_FC[FC_columns].sort_values(by='per_end_date'), 
    by='ticker', 
    left_on='date', 
    right_on='per_end_date', 
    direction='backward'
).sort_values(["ticker", "date"])


financial_ratios = pd.merge_asof(
    financial_ratios.sort_values(by='date'), 
    ZACKS_SHRS[['ticker', 'per_end_date', 'shares_out']].sort_values(by='per_end_date'), 
    by='ticker', 
    left_on='date', 
    right_on='per_end_date', 
    direction='backward'
).sort_values(["ticker", "date"])

financial_ratios.drop(columns=['per_end_date_x', 'per_end_date_y'], inplace=True)



In [13]:
financial_ratios

,ticker,date,adj_close,adj_volume,mkt_val,tot_debt_tot_equity,ret_invst,net_lterm_debt,tot_lterm_debt,eps_diluted_net,basic_net_eps,filing_date,shares_out
0,A,2017-01-03,43.83321,1739726.00000,17040.05000,0.46330,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1910,A,2017-01-04,44.40835,1821264.00000,17040.05000,0.46330,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2922,A,2017-01-05,43.88035,1503763.00000,17040.05000,0.46330,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3255,A,2017-01-06,45.24749,2883483.00000,17040.05000,0.46330,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4769,A,2017-01-09,45.38892,2575328.00000,17040.05000,0.46330,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1838184,ZWS,2024-06-24,29.95333,928354.00000,5780.16000,0.30700,1.61260,-0.20000,494.50000,0.19000,0.20000,2024-04-23,172.70000
1839390,ZWS,2024-06-25,29.57519,645262.00000,5780.16000,0.30700,1.61260,-0.20000,494.50000,0.19000,0.20000,2024-04-23,172.70000
1840544,ZWS,2024-06-26,29.05772,1058406.00000,5780.16000,0.30700,1.61260,-0.20000,494.50000,0.19000,0.20000,2024-04-23,172.70000
1840634,ZWS,2024-06-27,29.01791,1028996.00000,5780.16000,0.30700,1.61260,-0.20000,494.50000,0.19000,0.20000,2024-04-23,172.70000
